In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
from google.colab import drive
import numpy as np

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Define paths to save models in Google Drive
best_model_path_ijepa = '/content/drive/My Drive/best_ijepa_model_CNN.pth'
best_model_path_linear = '/content/drive/My Drive/best_linear_probing_model_CNN.pth'

In [4]:
class CNNEncoder(nn.Module):
    def __init__(self, embed_dim=128):
        super(CNNEncoder, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        # Set the expected size after the last convolutional layer
        self.fc_input_size = 128 * 1 * 1  # Output shape after conv layers
        self.fc = nn.Sequential(
            nn.Linear(self.fc_input_size, 256),  # Intermediate layer
            nn.ReLU(),
            nn.Linear(256, embed_dim)  # Final output layer
        )

    def forward(self, x):
        x = self.conv_layers(x)
        # print(f"Shape after conv layers: {x.shape}")
        x = x.view(x.size(0), -1)  # Flatten the tensor
        output = self.fc(x)
        return output


In [5]:

class ContextEncoder(CNNEncoder):
    def __init__(self, embed_dim=128):
        super(ContextEncoder, self).__init__(embed_dim)


In [6]:


class TargetEncoder(CNNEncoder):
    def __init__(self, embed_dim=128):
        super(TargetEncoder, self).__init__(embed_dim)

    def update_target_weights(self, context_encoder, decay=0.99):
        for target_param, context_param in zip(self.parameters(), context_encoder.parameters()):
            target_param.data = decay * target_param.data + (1.0 - decay) * context_param.data


In [7]:
class Predictor(nn.Module):
    def __init__(self, input_dim=256, output_dim=128):  # Add output_dim parameter
        super(Predictor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)  # Input dimension remains 256
        self.fc2 = nn.Linear(256, output_dim)  # Change output dimension to match actual_target_rep size

    def forward(self, context_rep, mask_token):
        x = torch.cat((context_rep, mask_token), dim=1)  # Shape [64, 256]
        x = F.relu(self.fc1(x))  # First fully connected layer
        x = self.fc2(x)  # Second fully connected layer
        return x

# Define the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Initialize the context encoder, target encoder, and predictor
context_encoder = ContextEncoder(embed_dim=128).to(device)
target_encoder = TargetEncoder(embed_dim=128).to(device)
predictor = Predictor(input_dim=256).to(device)

In [8]:
# L2 loss function
class IJEPALoss(nn.Module):
    def __init__(self):
        super(IJEPALoss, self).__init__()
        self.criterion = nn.MSELoss()

    def forward(self, pred_target_rep, actual_target_rep):
        return self.criterion(pred_target_rep, actual_target_rep)

In [9]:
def create_context_target_blocks(image, mask_ratio=0.25):
    B, C, H, W = image.shape
    mask_size = int(H * mask_ratio)

    # Select context and target regions
    context = image[:, :, :H - mask_size, :W - mask_size]
    target = image[:, :, mask_size:, mask_size:]

    # print(f"Created Context Shape: {context.shape}")
    # print(f"Created Target Shape: {target.shape}")

    return context.to(device), target.to(device)

In [10]:
# Pretraining with I-JEPA using EMA for the target encoder
def pretrain_ijepa(context_encoder, target_encoder, predictor, train_loader, start_epoch=0, epochs=200, lr=0.001, patience=5, decay=0.99):
    optimizer = optim.Adam(list(context_encoder.parameters()) + list(predictor.parameters()), lr=lr)
    criterion = IJEPALoss()

    best_loss = float('inf')
    patience_counter = 0

    for epoch in range(start_epoch, epochs):
        print(f"Starting Epoch {epoch + 1}/{epochs}")
        running_loss = 0.0
        for images, _ in train_loader:
            images = images.to(device)  # Ensure input images are on the correct device
            # print(f"Images Device: {images.device}")

            # Create context and target blocks
            context, target = create_context_target_blocks(images)
            #print(f"Context Device: {context.device}, Target Device: {target.device}")
            try:
                # Get representations from context and target encoders
                context_rep = context_encoder(context)
                # print(f"Context rep shape: {context_rep.shape}")

                actual_target_rep = target_encoder(target)
                # print(f"Target rep shape: {actual_target_rep.shape}")

                # Use mask tokens for prediction
                mask_token = torch.randn_like(context_rep).to(device)  # Ensure mask_token is the same shape as context_rep
                # print(f"Mask Token shape: {mask_token.shape}")

                # Ensure all tensors are on the same device before prediction
                assert context_rep.device == mask_token.device, "context_rep and mask_token are on different devices!"

                # Now pass both context_rep and mask_token to the predictor
                pred_target_rep = predictor(context_rep, mask_token)
                # print(f"Predicted Target Representation Device: {pred_target_rep.device}")  # Debugging print

                # Check if both tensors for loss are on the same device
                assert pred_target_rep.device == actual_target_rep.device, "pred_target_rep and actual_target_rep are on different devices!"

                # Compute the loss
                loss = criterion(pred_target_rep, actual_target_rep)

                # Backpropagation and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
            except Exception as e:
                print(f"Error encountered: {e}")

        avg_loss = running_loss / len(train_loader)
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}')

        # Update target encoder weights using EMA
        target_encoder.update_target_weights(context_encoder, decay=decay)

        # Early stopping and model saving logic
        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
            torch.save({
                'context_encoder': context_encoder.state_dict(),
                'target_encoder': target_encoder.state_dict(),
                'predictor': predictor.state_dict(),
            }, best_model_path_ijepa)
            print(f"Best I-JEPA model saved at epoch {epoch + 1} with loss {best_loss:.4f}")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered!")
                break

    print("I-JEPA pretraining complete!")

In [11]:
# DataLoader for CIFAR-10 with normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # Normalization based on CIFAR-10 dataset stats
])


In [12]:
# Load the full CIFAR-10 dataset
full_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)


100%|██████████| 170M/170M [00:17<00:00, 9.95MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [13]:
# Split the dataset into train, validation, and test sets
train_size = len(full_trainset) - 10000  # 5000 for validation + 5000 for testing
val_size = 5000
test_size = 5000

In [14]:
trainset, valset, testset = random_split(full_trainset, [train_size, val_size, test_size])


In [15]:

train_loader = DataLoader(trainset, batch_size=64, shuffle=True)
val_loader = DataLoader(valset, batch_size=64, shuffle=False)
test_loader = DataLoader(testset, batch_size=64, shuffle=False)

In [31]:
# Pretrain the model using I-JEPA
pretrain_ijepa(context_encoder, target_encoder, predictor, train_loader)

Starting Epoch 1/200
Epoch 1/200, Loss: 0.0181
Best I-JEPA model saved at epoch 1 with loss 0.0181
Starting Epoch 2/200
Epoch 2/200, Loss: 0.0140
Best I-JEPA model saved at epoch 2 with loss 0.0140
Starting Epoch 3/200
Epoch 3/200, Loss: 0.0129
Best I-JEPA model saved at epoch 3 with loss 0.0129
Starting Epoch 4/200
Epoch 4/200, Loss: 0.0120
Best I-JEPA model saved at epoch 4 with loss 0.0120
Starting Epoch 5/200
Epoch 5/200, Loss: 0.0113
Best I-JEPA model saved at epoch 5 with loss 0.0113
Starting Epoch 6/200
Epoch 6/200, Loss: 0.0107
Best I-JEPA model saved at epoch 6 with loss 0.0107
Starting Epoch 7/200
Epoch 7/200, Loss: 0.0101
Best I-JEPA model saved at epoch 7 with loss 0.0101
Starting Epoch 8/200
Epoch 8/200, Loss: 0.0096
Best I-JEPA model saved at epoch 8 with loss 0.0096
Starting Epoch 9/200
Epoch 9/200, Loss: 0.0091
Best I-JEPA model saved at epoch 9 with loss 0.0091
Starting Epoch 10/200
Epoch 10/200, Loss: 0.0087
Best I-JEPA model saved at epoch 10 with loss 0.0087
Startin

In [16]:
# Load the pre-trained model checkpoint
checkpoint = torch.load(best_model_path_ijepa, map_location=device)

# Load full state_dicts directly, without filtering 'fc' layers
context_encoder.load_state_dict(checkpoint['context_encoder'], strict=False)
target_encoder.load_state_dict(checkpoint['target_encoder'], strict=False)
predictor.load_state_dict(checkpoint['predictor'])

<ipython-input-16-5882530dcaaa>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(best_model_path_ijepa, map_location=device)


<All keys matched successfully>